# Hyperprophet Demo

Hyperprophet is a service to compute thousands of concurrent time-series forecasts in minutes using the familiar Prophet API.

In [ ]:
from hyperprophet import Prophet
import pandas as pd

In [ ]:
url = "..."

In [ ]:
df = pd.read_csv(url)

In [ ]:
m = Prophet()